In [2]:
import numpy as np
from time import time
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import pdspy.modeling as modeling
import os
import pickle
import george
from george import kernels
from scipy.optimize import minimize

%matplotlib inline

In [15]:
dictionary=np.load("./gmd/dictionary.npy")
with open ('./gmd/cubefull.txt', 'rb') as fp:
    cube = np.array(pickle.load(fp))[:,100:500]

In [47]:
rname="orig"

# load cube
with open ("./gmd/cubefull.txt","rb") as fp:
	cube=np.array(pickle.load(fp))[:,100:500]

# load pca weight data generated by get_pca_weights.py
with open ("./gmd/"+rname+"_parvals.txt","rb") as fp:
	param_vector=pickle.load(fp)
with open ("./gmd/"+rname+"_weights.txt","rb") as fp:
	weights_pw=pickle.load(fp)
with open ("./gmd/"+rname+"_eigenseds.txt","rb") as fp:
	eigenseds=pickle.load(fp)
with open ("./gmd/"+rname+"_mean.txt","rb") as fp:
	pcamean=pickle.load(fp)
print("pca weights loaded from "+rname)


pca weights loaded from orig


In [48]:
np.array(cube).shape

(3850, 400)

In [31]:
weights_pm=np.transpose(np.array(weights_pw))

In [80]:
points=5
comp_param=np.zeros((points,points))
comp_obs_chisq=np.zeros((points,points))
comp_obs_weights=np.zeros((points,points))

for i in range(points):
    for j in range(points):
        s=[cube[i],cube[j]]
        p=[np.array(param_vector[i]),np.array(param_vector[j])] ##rescale parameters so this isn't so temp-focused
        comp_param[i][j]=np.linalg.norm(p[0]-p[1])
        comp_obs_weights[i][j]=np.linalg.norm(s[0]-s[1])
        comp_obs_chisq[i][j]=np.sum((s[0]-s[1])**2)

In [81]:
print(comp_param)
print(comp_obs_weights)
print(comp_obs_chisq)

[[   0.         1207.16802727  320.49397858  758.22447278  207.41887726]
 [1207.16802727    0.         1527.04130319  452.42093932 1412.22745633]
 [ 320.49397858 1527.04130319    0.         1078.66470936  115.69170726]
 [ 758.22447278  452.42093932 1078.66470936    0.          964.69463836]
 [ 207.41887726 1412.22745633  115.69170726  964.69463836    0.        ]]
[[ 0.         46.02672195 49.51483917 54.68814468 65.35608673]
 [46.02672195  0.         30.72829819 18.68395996 46.49970245]
 [49.51483917 30.72829819  0.         27.69950676 22.57680893]
 [54.68814468 18.68395996 27.69950676  0.         44.27814865]
 [65.35608673 46.49970245 22.57680893 44.27814865  0.        ]]
[[   0.         2118.45898438 2451.71923828 2990.79272461 4271.41796875]
 [2118.45898438    0.          944.22839355  349.09039307 2162.22216797]
 [2451.71923828  944.22839355    0.          767.26269531  509.71231079]
 [2990.79272461  349.09039307  767.26269531    0.         1960.55419922]
 [4271.41796875 2162.22216